# Measure without a Loop

If you have a parameter that returns a whole array at once, often you want to measure it directly into a DataSet.

This shows how that works in QCoDeS

In [ ]:
%matplotlib nbagg
import qcodes as qc
# import dummy driver for the tutorial
from qcodes.tests.instrument_mocks import DummyInstrument
from qcodes.instrument.mock import ArrayGetter

dac1 = DummyInstrument(name="dac")
dac2 = DummyInstrument(name="dac2")
# the default dummy instrument returns always a constant value, in the following line we make it random 
# just for the looks 💅
import random
dac2.dac2.get =  lambda: random.randint(0,100)

# The station is a container for all instruments that makes it easy 
# to log meta-data
station = qc.Station(dac1, dac2)

## Instantiates all the instruments needed for the demo

For this tutorial we're going to use the regular parameters (c0, c1, c2, vsd) and ArrayGetter, which is just a way to construct a parameter that returns a whole array at once out of simple parameters, as well as AverageAndRaw, which returns a scalar *and* an array together.

### Only array output
The arguments to Measure are all the same actions you use in a Loop.
If they return only arrays, you will see exactly those arrays (with their setpoints) in the output DataSet

In [ ]:
data = qc.Measure(
    qc.Task(dac1.dac1.set, 0),
    ArrayGetter(dac1.dac2, dac1.dac3.sweep(-10,10,0.2), 0.001),
    qc.Task(dac1.dac1.set, 2),
    ArrayGetter(dac1.dac2, dac1.dac3.sweep(-10,10,0.2), 0.001),
).run()